# Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [4]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [5]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [6]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote,Simple 1,Simple 2,Simple 3,Ex 1.1,Ex 1.1 modified,Ex 1.2,Ex A.1,Ex 1.3,Ex 2.1,Ex 2.2,...,Ex B.3,Ex C.1,Ex C.2,Ex D.1,Ex Alg1.3,Ex Alg1.4,Ex Alg1.5,Ex Alg2.1,Ex Alg A.1,Ex Alg A.2
subtest,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,...,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"Phragmen(marginal_slate, clear=True, redist=True)",1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.5,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Overall Performance

In [7]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)    27.000000
ExactTotalUtilityMaximization(utility_transform=None)     26.958333
Phragmen(marginal_slate, clear=True, redist=True)         26.000000
dtype: float64

In [8]:
pd.DataFrame(np.trunc(df.values), index=df.index, columns=df.columns).sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)    27.0
ExactTotalUtilityMaximization(utility_transform=None)     26.0
Phragmen(marginal_slate, clear=True, redist=True)         25.0
dtype: float64

## Differential Test Performance

In [9]:
non_uniform_columns = df.loc[:, df.nunique() > 1]
non_uniform_columns


vote,Ex A.1,Ex 1.3,Ex 3.1
subtest,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.000000
ExactTotalUtilityMaximization(utility_transform=None),1.0,1.0,0.958333
"Phragmen(marginal_slate, clear=True, redist=True)",0.5,0.5,1.000000


These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [10]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)



Counts for level 0:
vote
Ex A.1    1
Ex 1.3    1
Ex 3.1    1
Name: count, dtype: int64

Counts for level 1:
subtest
Individual Pareto Efficiency    3
Name: count, dtype: int64


## Axiom Pass Rate

In [11]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

C:\Users\aaron\AppData\Local\Temp\ipykernel_13724\1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Individual Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.000000
ExactTotalUtilityMaximization(utility_transform=None),0.998457
"Phragmen(marginal_slate, clear=True, redist=True)",0.962963


# Debuggign

In [12]:
# alg = GreedyTotalUtilityMaximization()
# alg = SequentialPhragmenMinimax()
alg = ExactTotalUtilityMaximization()
# alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))

In [13]:
df1 = df.loc[alg.name,:].unstack()
df1

subtest,Individual Pareto Efficiency
vote,
Ex 1.1,1.000000
Ex 1.1 modified,1.000000
Ex 1.2,1.000000
Ex 1.3,1.000000
Ex 2.1,1.000000
Ex 2.2,1.000000
Ex 3.1,0.958333
Ex 4.1,1.000000
Ex 4.2,1.000000


In [14]:
# case = rated_vote_cases["Ex 1.1 modified"]
case = rated_vote_cases["Ex 3.1"]
# case = rated_vote_cases["Ex 4.3"]
# case = rated_vote_cases["Simple 3"]
# axiom = CoverageAxiom()
# axiom = HappiestParetoAxiom()
axiom = IndividualParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
# axiom = NonRadicalTotalUtilityAxiom()

case.rated_votes

,s1,s2,s3,s4,s5,s6
0,2,0,1,0,0,0
1,1,2,0,0,0,0
2,0,1,2,0,0,0
3,0,0,0,2,0,1
4,0,0,0,1,2,0
5,0,0,0,0,1,2


In [15]:
df.loc[:,df.columns.get_level_values(1) == axiom.name].iloc[:,10:-10]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote,Ex 3.1,Ex 4.1,Ex 4.2,Ex 4.3,Ex 4.4,Ex B.1,Ex B.2
subtest,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.000000,1.0,1.0,1.0,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=None),0.958333,1.0,1.0,1.0,1.0,1.0,1.0
"Phragmen(marginal_slate, clear=True, redist=True)",1.000000,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s1', 's2', 's5']

In [17]:
# axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)

In [18]:
num_aug_cases = 24
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases[:num_aug_cases]):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [19]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied


In [20]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


IndexError: single positional indexer is out-of-bounds

In [ ]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2,s3,s4,s5,s6
0,-107.880498,-184.719314,-143.090445,-14.018711,-107.420886,-12.285417
1,-107.880498,-184.719314,-143.090445,-14.018711,-107.420886,-12.285417
2,-107.880498,-184.719314,-143.090445,-14.018711,-107.420886,-12.285417
3,-107.880498,-184.719314,-143.090445,-14.018711,-107.420886,-12.285417
4,-107.880498,-184.719314,-143.090445,-14.018711,-107.420886,-12.285417
5,-107.880498,-184.719314,-143.090445,-14.018711,-107.420886,-12.285417


In [ ]:
f0.assignments

,candidate_id
0,s1
1,s1
2,s4
3,s4
4,s4
5,s6


In [ ]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's4', 's6'],
   candidate_id
 0           s1
 1           s1
 2           s4
 3           s4
 4           s4
 5           s6)

In [ ]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

NameError: name 'f0' is not defined

In [ ]:
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['HappiestParetoAxiom',
 'NonRadicalMinUtilityAxiom',
 'NonRadicalTotalUtilityAxiom',
 'IndividualParetoAxiom',
 'MinimumAndTotalUtilityParetoAxiom',
 'CoverageAxiom']

# Scratch

In [ ]:
from generative_social_choice.slates.voting_utils import filter_candidates_by_individual_pareto_efficiency
from generative_social_choice.slates.voting_algorithms import RatedVoteCase

# case = rated_vote_cases["Simple 3"]
case = RatedVoteCase(
    name="BLAH",
    rated_votes=pd.DataFrame([
            [3, 1, 2],  # Voter 0 prefers candidate 0
            [1, 3, 2],  # Voter 1 prefers candidate 1  
            [1, 2, 3]   # Voter 2 prefers candidate 2
        ], columns=["A", "B", "C"]),
        # ], columns=["A", "B", "C", "D"]),
    slate_size=2
)

filter_candidates_by_individual_pareto_efficiency(case.rated_votes)


,A,B,C
0,3,1,2
1,1,3,2
2,1,2,3
